In [1]:
import skimage.measure as msr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
from PIL import Image, ImageCms

import glob
import os
from pathlib import Path
import imageio
import cv2

In [2]:
# For importing wmark.py from parent directory
# import sys
# sys.path.append("..")

In [3]:
# Importing class from wmark.py
from wmark import WaterMark

In [4]:
# Source Directory
srcFolder = 'TestSet/'
# Source Path
srcPth = Path(srcFolder).resolve()

# Destination Directory
# dstFolder = Path(srcFolder+'/marked/').resolve()
# Creating folder for marked images
# Path(dstFolder).mkdir(exist_ok=True)
# Destination Path
# dstPath = Path(dstFolder).resolve()

In [13]:
# Creating empty DataFrame that will fill up after the loop
seedDataframe = pd.DataFrame()

for i in range(1, 5):
    
    wObject = WaterMark(i)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 1])
    counter = 0
    
    for img in imgsSource:
            
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 0] = metricMarked
        
        counter += 1
      
        finalMetric = pd.DataFrame(results, columns = ["Seed"])
        
    seedDataframe = pd.concat([seedDataframe, finalMetric], axis = 1)

In [14]:
seedDataframe

,Seed,Seed,Seed,Seed
0,0.684302,0.726550,0.693043,0.694871
1,0.765136,0.773062,0.779924,0.730109
2,0.879576,0.918984,0.904692,0.905918
3,0.754294,0.767509,0.807421,0.751422
4,0.946265,0.952480,0.956471,0.944764


In [8]:
# Creating empty DataFrame that will fill up after the loop
impactFactorDataframe = pd.DataFrame()
     
# Generating random seed as integer up to 10000
randomNums = np.random.randint(10000, size=(1))
randomNum = randomNums[0]
print(randomNum)

wObject = WaterMark(randomNum)

# All TIF files in the src_path are now imgs
imgsSource = srcPth.glob('*.tif')
    
# Counting the number of images in src_folder
list = os.listdir(srcFolder)
# -1 because there is one hidden file in directory
numImages = len(list) - 1

# Creating zeros Numpy Array and setting counter for results
results = np.zeros([numImages*5, 2])
counter = 0
    
for img in imgsSource:
    
    impFctRange = range(0, 10001, 2500)
        
    for impFct in impFctRange:
        
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Mark image with different Impact Factor values
        imgMarked = wObject.embedMark(imgOriginal, factor = impFct)
        
        # GCR Masking
        imgGCR = WaterMark.gcrMasking(imgOriginal, imgMarked)
            
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 0] = metricMarked
        
        metricGCR = wObject.decodeMark(imgGCR, 'CORR')
        results[counter, 1] = metricGCR
            
        print(counter)
            
        metricValues = ["Marked", "GCR"] 
        impactFactorFinalMetric = pd.DataFrame(results, columns = [metricValues])
            
        counter += 1
            
    impactFactorDataframe = pd.concat([impactFactorDataframe, impactFactorFinalMetric], axis = 1)

2918
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [9]:
impactFactorDataframe

,Marked,GCR,Marked,GCR,Marked,GCR,Marked,GCR,Marked,GCR
0,0.256293,0.256310,0.256293,0.256310,0.256293,0.256310,0.256293,0.256310,0.256293,0.256310
1,0.783156,0.636296,0.783156,0.636296,0.783156,0.636296,0.783156,0.636296,0.783156,0.636296
2,0.791162,0.664852,0.791162,0.664852,0.791162,0.664852,0.791162,0.664852,0.791162,0.664852
3,0.792722,0.660015,0.792722,0.660015,0.792722,0.660015,0.792722,0.660015,0.792722,0.660015
4,0.793106,0.651114,0.793106,0.651114,0.793106,0.651114,0.793106,0.651114,0.793106,0.651114
5,0.000000,0.000000,0.278727,0.279333,0.278727,0.279333,0.278727,0.279333,0.278727,0.279333
6,0.000000,0.000000,0.809315,0.758820,0.809315,0.758820,0.809315,0.758820,0.809315,0.758820
7,0.000000,0.000000,0.823059,0.778129,0.823059,0.778129,0.823059,0.778129,0.823059,0.778129
8,0.000000,0.000000,0.826614,0.775561,0.826614,0.775561,0.826614,0.775561,0.826614,0.775561
9,0.000000,0.000000,0.827841,0.772269,0.827841,0.772269,0.827841,0.772269,0.827841,0.772269


In [10]:
impactFactorFinalMetric

,Marked,GCR
0,0.256293,0.256310
1,0.783156,0.636296
2,0.791162,0.664852
3,0.792722,0.660015
4,0.793106,0.651114
5,0.278727,0.279333
6,0.809315,0.758820
7,0.823059,0.778129
8,0.826614,0.775561
9,0.827841,0.772269


In [ ]:
impactFactorFinalMetric.loc[[0, 1, 2, 3, 4], :]

In [ ]:
impactFactorFinalMetric.loc[[5, 6, 7, 8, 9], :]

In [ ]:
impactFactorFinalMetric.loc[[10, 11, 12, 13, 14], :]

In [ ]:
# Creating empty DataFrame that will fill up after the loop
metricDataframe = pd.DataFrame()
for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    randomNums = np.random.randint(10000, size=(1))
    randomNum = randomNums[0]
    print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 10])
    counter = 0
        
    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgMarked.shape[0]
        # Height
        ly = imgMarked.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgMarked[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgMarked, (width, height))

        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgMarked.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgMarked, M, (cols, rows))    

        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgMarked.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgMarked, M, (cols, rows))
          
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgMarked, -1, kernel)  
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgMarked, (kernelX, kernelY), cv2.BORDER_DEFAULT)
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgMarked.size)
        gauss = gauss.reshape(
            imgMarked.shape[0], imgMarked.shape[1], imgMarked.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgMarked, gauss)
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        results[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        results[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        results[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        results[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        results[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        results[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        results[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        results[counter, 7] = metricNoised
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 8] = metricMarked
        
        metricImpactFactor = ImpactFactor
        results[counter, 9] = metricImpactFactor
        
        counter += 1
        
        metricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        metricSeeds = [f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}"]
        
        finalMetric = pd.DataFrame(results, columns = [metricSeeds, metricValues])
        
    metricDataframe = pd.concat([metricDataframe, finalMetric], axis = 1)

In [ ]:
# Creating empty DataFrame that will fill up after the loop
gcrMetricDataframe = pd.DataFrame()

for i in range(1, 2):
    
    # Using seed from previous cell
    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    gcrResults = np.zeros([numImages, 10])
    counter = 0
        
    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        # GCR Masking
        imgGCR = WaterMark.gcrMasking(imgOriginal, imgMarked)
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgGCR.shape[0]
        # Height
        ly = imgGCR.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgGCR[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgGCR, (width, height))

        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgGCR.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgGCR, M, (cols, rows))    

        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgGCR.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgGCR, M, (cols, rows))
    
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgGCR, -1, kernel)
       
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgGCR, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgGCR.size)
        gauss = gauss.reshape(
            imgGCR.shape[0], imgGCR.shape[1], imgGCR.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgGCR, gauss)
      
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        gcrResults[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        gcrResults[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        gcrResults[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        gcrResults[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        gcrResults[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        gcrResults[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        gcrResults[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        gcrResults[counter, 7] = metricNoised
        
        metricGCR = wObject.decodeMark(imgGCR, 'CORR')
        gcrResults[counter, 8] = metricGCR
        
        metricImpactFactor = ImpactFactor
        gcrResults[counter, 9] = metricImpactFactor
        
        counter += 1
        
        gcrMetricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        gcrMetricSeeds = [f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}"]
           
        gcrFinalMetric = pd.DataFrame(gcrResults, columns = [gcrMetricSeeds, gcrMetricValues])
        
    gcrMetricDataframe = pd.concat([gcrMetricDataframe, gcrFinalMetric], axis = 1)

In [ ]:
# Show Pandas DataFrame
metricDataframe

In [ ]:
# Show Pandas DataFrame
gcrMetricDataframe

In [ ]:
gcrMetricDataframe.boxplot(rot=90)

In [ ]:
describeDataframe = metricDataframe.describe()

In [ ]:
gcrdescribeDataframe = gcrMetricDataframe.describe()

In [ ]:
compareDataframe = describeDataframe.join(gcrdescribeDataframe)

In [ ]:
comparemeanDataframe = compareDataframe.iloc[1]

In [ ]:
comparemeanDataframeUnstacked = comparemeanDataframe.unstack(level = 0)

In [ ]:
comparemeanDataframeUnstacked

In [ ]:
# Colors: https://github.com/matplotlib/matplotlib/blob/38be7aeaaac3691560aeadafe46722dda427ef47/lib/matplotlib/mpl-data/stylelib/fivethirtyeight.mplstyle

plt.style.use('fivethirtyeight')

# Creating graph
fig, ax1 = plt.subplots(figsize=(12,8))

# Setting X-axis and first Y-axis
ax1.set_xlabel('Impact Factor')
ax1.set_ylabel('PSNR', color = '#008fd5')

# Creating a second Y-axis that shares the same X-axis
ax2 = ax1.twinx()  
ax2.set_ylabel('SSIM', color = '#fc4f30')

# Show grid
ax1.grid(True)
ax2.grid(True)

# Setting different colors for each variable
ax1.plot(comparemeanDataframeUnstacked['2982'], color = '#008fd5')
ax2.plot(comparemeanDataframeUnstacked['9721'], color = '#fc4f30')

In [ ]:
stackedDataframe.plot.hist(legend = True)